# Intersight Python SDK

If you are familiar Python, you can take advantage of the Intersight Python SDK available for install from the [Python Package Index](https://pypi.org/project/intersight/).

![Python SDK for Cisco Intersight](images/python_sdk_pip.png)

## Installation

Notice the `pip install intersight` command. That's the command used to install the Python SDK in your environment.  Be sure you are using Python >= 3.7, earlier versions of Python are not supported.

In [ ]:
pip install intersight


> Be sure to uninstall any conflicting versions of the SDK if you have previous installs.  You can check installed versions with pip list.

In [ ]:
pip list | grep -i intersight

> If you see Intersight-OpenAPI installed, you can run `pip unistall Intersight-OpenAPI`

## Using the Intersight Python SDK

To use the SDK, you must import required modules.  You can view help once you've imported any required modules.


In [ ]:
import intersight

# View the help
help(intersight)


### API Keys and User Authentication

Now let's use the Intersight Python SDK to connect to our Intersight environment.  First, you'll need an Intersight API Key ID and secret (private) key from your Intersight account.  From the Settings menu in Intersight, Select API Keys and Generate API Key.

![Generate API Key](images/generate_api_key.png)

A version 3 key can be used with the SDK and is recommended for long term compatibility with Intersight's API.

![Version 3 key](images/version_3_key.png)

The Generated API Key ID can be copied to the clipboard and used in API authentication or can be placed into a file.  The example below uses a file for lookup so the ApiKeyId is not hardcoded in this notebook.

Be sure to save the Secret (private) Key to a local file that only you have access to (a download option is provided by Intersight).

![Download Secret Key](images/secret_key_download.png)

### Intersight API Configuration

We'll create an API client and use it to connect with Intersight.

> Note that in DevNet labs the values below can be used as is.  For other environments, be sure to enter the values for your account in the `key_id` variable below as well as the `private_key_path`.  Also be sure that you are using v3 API keys from your Intersight account.  

In [ ]:
import intersight

with open('./ApiKeyIdFile', 'r') as file:
    api_key_id = file.read().rstrip()

configuration = intersight.Configuration(
    signing_info=intersight.HttpSigningConfiguration(
        # key_id='596cc79e5d91b400010d15ad/60ede8d07564612d3335edac/623354c07564612d33d9068d',
        key_id=api_key_id,
        private_key_path='./SecretKey.txt',
        signing_scheme=intersight.signing.SCHEME_HS2019,
        signed_headers=[intersight.signing.HEADER_HOST,
                        intersight.signing.HEADER_DATE,
                        intersight.signing.HEADER_DIGEST,
                        intersight.signing.HEADER_REQUEST_TARGET
                        ]
    )
)
api_client = intersight.ApiClient(configuration)


### Intersight Python SDK Query Examples

Now that you are authenticated, let's use the SDK to query server and related resources in Intersight.  You can read more on [Intersight's API query language](https://www.intersight.com/apidocs/introduction/query/), and to get an idea of the resources used by the API you can look at the URIs your browser uses (compute/physical-summaries below).

![Browser Compute Summary](images/browser_compute_phys_summary.png)

The Python example below sets up a query filter to get a count of servers and then uses the returned count to page through all servers in the account.  When getting server summaries, the select query parameter is used to only return certain server attributes.

In [ ]:
import intersight.api.compute_api

# Create physical summary (server) class instance
api_instance = intersight.api.compute_api.ComputeApi(api_client)

# Find the count of servers in the account
server_query = api_instance.get_compute_physical_summary_list(count=True)
print(server_query)

# Intersight limits the number of items returned.  Page through returned results and select Name, Model, Serial
per_page = 50
query_select = "Name,Model,Serial"
for i in range(0, server_query.count, per_page):
    page_query = api_instance.get_compute_physical_summary_list(top=per_page, skip=i, select=query_select)
    print(page_query)


### Detailed Inventory

Building on the server inventory example above, we'll now look at Intersight's resource model to get information on server components like physical disks.  We can search in the API for specific disk Model numbers or search for Models that contain a certian substring.  Below we will search for physical disks Model numbers that contain a certain substring. 

In [ ]:
import intersight.api.storage_api

# Create storage class instance
api_instance = intersight.api.storage_api.StorageApi(api_client)

# Setup query options
query_filter = "contains(Model,'UCS')"
query_select = "Dn,Model,RegisteredDevice,RunningFirmware"
query_expand = "RegisteredDevice($select=DeviceHostname),RunningFirmware($select=Version)"

# Get physcial disks that contain the Model substring defined above
storage_query = api_instance.get_storage_physical_disk_list(filter=query_filter, select=query_select, expand=query_expand)
print(storage_query)


## Configure Resources

### Create a new BIOS Policy

In [ ]:
from intersight.api import bios_api
from intersight.model.bios_policy import BiosPolicy
from intersight.api import organization_api
from intersight.model.organization_organization_relationship import OrganizationOrganizationRelationship

organization_instance = organization_api.OrganizationApi(api_client)
api_response = organization_instance.get_organization_organization_list(filter="Name eq 'Demo-DevNet'")
organization = OrganizationOrganizationRelationship(moid=api_response.results[0].moid,
                                                    object_type="organization.Organization",
                                                    class_id="mo.MoRef",
                                                    )
print(organization)

bios_policy = BiosPolicy()
bios_policy.name = "CLive2024"
bios_policy.organization = organization
bios_policy.cpu_energy_performance = "performance"
bios_policy.cpu_performance = "hpc"

# Create a bios.Policy resource
bios_policy_instance = bios_api.BiosApi(api_client)
api_response = bios_policy_instance.create_bios_policy(bios_policy)
print("Name: %s, CPU Energy %s, CPU Perf %s" % (api_response.name, api_response.cpu_energy_performance, api_response.cpu_performance))


### Update a BIOS Policy

In [ ]:
bios_policy.cpu_performance = "enterprise"
api_response = bios_policy_instance.update_bios_policy(moid=api_response.moid, bios_policy=bios_policy)
print("Name: %s, CPU Energy %s, CPU Perf %s" % (api_response.name, api_response.cpu_energy_performance, api_response.cpu_performance))

### Delete a BIOS Policy

In [ ]:
api_response = bios_policy_instance.delete_bios_policy(moid=api_response.moid)
print(api_response)

## Additional Examples

Several additional examples of using the SDK are on GitHub in the [Intersight Python Utilties repo](https://github.com/CiscoDevNet/intersight-python-utils).  The repo also contains a credentials.py module to simplify authentication across all of the example scripts.